# Download Zoom Cloud Recording

A sample notebook which illustrates how to download a cloud recording from Zoom.

Imports:

In [ ]:
import requests #to get data
from bs4 import BeautifulSoup #to parse
import string #create valid filename
import time #timestamp for output filename

URL of cloud recording:

In [5]:
url = "https://zoom.us/rec/share/_PUT_DETAILS_HERE"

A function by [seanh](https://gist.github.com/seanh/93666) to create a valid filename:

In [6]:
#source: https://gist.github.com/seanh/93666
def format_filename(s):
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    filename = ''.join(c for c in s if c in valid_chars)
    filename = filename.replace(' ','_') # I don't like spaces in filenames.
    return filename

Get the actual video URL and download to current folder:

In [7]:
with requests.Session() as s: #so that cookies are kept along requests
    raw = s.get(url)
    parsed = BeautifulSoup(raw.text, 'html.parser')   #parse  
     #turns out in the original HTML response the actual URL is nested in <source> element inside of <video> element
    vurl = parsed.find('video').find_all('source')[1].get('src')
    # get video data
    vid = s.get(vurl, stream=True, timeout=20, headers={'referer':'https://zoom.us'}) #referer is needed because note included in requests session
    ofn = format_filename(parsed.find('title').string) + str(time.time()) + '.mp4' #create valid timestamped filename for output
    
    #write output to file:
    with open(ofn, 'wb') as f:
        for data in vid.iter_content(chunk_size=1024, decode_unicode=False):
            f.write(data)